# 3.1 Data transferred and ingested in butler and access to data

Get the same ras as we did on the summit

In [ ]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry
import sqlalchemy

repo = '/repo/embargo'
instrument = 'LATISS'
# raw all colleciton contains all images 
collections=['LATISS/raw/all']
butler = Butler(repo, collections=collections, instrument=instrument)


In [ ]:

dt='raw'
where="exposure.day_obs = 20230511"
datasetRefs = list(registry.queryDatasets(datasetType=dt,where=where))

print(f"There are {len(datasetRefs)} {dt} in collection: {collections} in: {repo} for {where}")

count = 0
for ref in datasetRefs[0], datasetRefs[-1]:
    count = count + 1
    print( ref.dataId['detector'], ref.dataId['exposure'])



In [ ]:
#Make a map of thoose for look up below .. 
dslookup = {}
for ref in datasetRefs:
    dslookup[ref.dataId['exposure']] = ref

### latency ..
"Measure the delays between completion of readout for each image and the ingestion time recorded in the Butler Registry."
The end of readou is the exposure.timespan.end.
Will have to lookup the ingest time usinf a sql query sine is not exposed directly in butler.

In [ ]:
from astropy import time

res = butler.registry.queryDimensionRecords('exposure', where=where, instrument=instrument)
print (f"Got Metadata for {res.count()} exposures")
times = []
min = 100
max = 0
for count,dimensionRecord in enumerate(res):
    datasetRef = dslookup[dimensionRecord.id]
    uuid = datasetRef.id
    # the ID in the dimesnion record is the exposure ID (dimensionRecord.id)
    # the ID in the oga.dataset is the UUID from the datasetRef 
    # hence the lookup map created in teh cell above is used here to lookup the UUID
    
    q_string = f"SELECT ingest_date FROM oga.dataset WHERE id='{uuid}'"
    with registry._db.query(sqlalchemy.text(q_string)) as q:
        intime = time.Time(q.all()[0][0])
        
    diff = (intime - dimensionRecord.timespan.end ).to('minute')
    if diff.value < min:
        min = diff.value
    if diff.value > max:
        max = diff.value 
    #print(r.id,r.timespan.end, intime, diff )
    times.append(diff.value)
    
import numpy as np
print (f"The min time was {min} max time was {max} minutes")
print (f"Time between shutter close and ingest mean:{np.mean(times)}, std:{np.std(times)}, var:{np.var(times)} minutes")

In [ ]:
#3.2 LFA replication to USDF
Look in EFD for events with Large Files.
Get a few of the files randomly 

In [2]:
from lsst_efd_client import EfdClient, resample
loc = 'usdf'

client = EfdClient(f'{loc}_efd')  
client.output = 'dataframe'
cl=client.influx_client
day = '2023-04-20'

query=f'''SELECT url, id FROM "lsst.sal.Electrometer.logevent_largeFileObjectAvailable" WHERE time >  '{day}' '''
#query 'SELECT "url", "id" FROM "efd"."autogen"."lsst.sal.Electrometer.logevent_largeFileObjectAvailable" WHERE "time" >  2023-04-20T '
result = await cl.query(query)


In [13]:
print(f"Got {len(result)} Large files availble since {day} at {loc}")
# KT says we  need this next line
os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"
from lsst.resources import ResourcePath
import random
for f in range(0,5):
   pos = random.randrange(0,len(result))
   file_uri = ResourcePath(result.url[pos].replace("https://s3.cp.lsst.org/","s3://rubin:")  )  
   print (f"result[{pos}] {result.url[pos]}")
   print (f"Access {file_uri}")
   contents = file_uri.read()
   print (f"Object is {len(contents)} bytes. {contents[0:80]}")


Got 173 Large files availble since 2023-04-20 at usdf
result[168] https://s3.cp.lsst.org/rubinobs-lfa-cp/Electrometer:201/fits/2023/05/02/Electrometer:201_fits_2023-05-03T01:57:36.248.fits
Access s3://rubin:rubinobs-lfa-cp/Electrometer:201/fits/2023/05/02/Electrometer:201_fits_2023-05-03T01:57:36.248.fits
Object is 8640 bytes. b'SIMPLE  =                    T / conforms to FITS standard                      '
result[51] https://s3.cp.lsst.org/rubinobs-lfa-cp/Electrometer:201/fits/2023/05/02/Electrometer:201_fits_2023-05-02T23:45:47.304.fits
Access s3://rubin:rubinobs-lfa-cp/Electrometer:201/fits/2023/05/02/Electrometer:201_fits_2023-05-02T23:45:47.304.fits
Object is 8640 bytes. b'SIMPLE  =                    T / conforms to FITS standard                      '
result[73] https://s3.cp.lsst.org/rubinobs-lfa-cp/Electrometer:201/fits/2023/05/02/Electrometer:201_fits_2023-05-03T00:11:00.247.fits
Access s3://rubin:rubinobs-lfa-cp/Electrometer:201/fits/2023/05/02/Electrometer:201_fits_2023-0